In [1]:
import os 
os.chdir('..')

In [2]:
from app.inference import infer_tuned, infer_main

from app.models import get_model, get_tokenizer, get_tuning_tokenizer, get_tuning_model

In [3]:
model_main = get_model()
tokenizer_main = get_tokenizer()

/opt/homebrew/Caskroom/miniforge/base/envs/mtpqt/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/mtpqt/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
infer_main(["My name is Noor", "I like the color Black Pyramids"],model_main, tokenizer_main, max_length=64)

[{'Noor': {'index': [3], 'entity': 'person'}},
 {'Black Pyramids': {'index': [4, 5], 'entity': 'artwork'}}]

In [5]:
"I will go to work at 5:00 AM".split()

['I', 'will', 'go', 'to', 'work', 'at', '5:00', 'AM']

In [4]:
from app.models import get_model, get_tokenizer, get_tuning_tokenizer, get_tuning_model
from app.training.training_routine import get_trainer, prepare_data

model_tuned = get_tuning_model(path = "models/checkpoint-248")
tokenizer_tuned = get_tuning_tokenizer()

/opt/homebrew/Caskroom/miniforge/base/envs/mtpqt/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
infer_tuned(["I like the 5 kgs", "I will go to the office tomorrow at 5 o'clock"], model_tuned, tokenizer_tuned, max_length=64)

/opt/homebrew/Caskroom/miniforge/base/envs/mtpqt/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[{'5': {'index': [3], 'entity': 'Quantity'},
  'kgs': {'index': [4], 'entity': 'Units'}},
 {'5': {'index': [8], 'entity': 'Quantity'},
  "o ' clock": {'index': [9, 10, 11], 'entity': 'Units'}}]

In [7]:
from app.training.training_helpers import compute_metrics_2

train_set, test_set, _, _ = prepare_data(tokenizer_tuned, re_process=False)
trainer = get_trainer(model_tuned, train_set, test_set, cuda=False, batch_size=2, compute_metrics=compute_metrics_2)

2024-08-13 09:51:45,044 - app.logger - WARNING - max_length is None, setting it to 64
2024-08-13 09:51:45,064 - app.logger - WARNING - max_length is None, setting it to 64


In [8]:
"I like the 5 kgs".index("kgs")

13

In [9]:
trainer.evaluate()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/30 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       B-Age       1.00      1.00      1.00         6
    B-Colors       0.75      0.38      0.50         8
  B-Currency       0.57      0.50      0.53        16
     B-Dates       0.79      0.65      0.71        17
    B-Prices       0.85      0.96      0.90        23
  B-Quantity       0.89      0.74      0.81        34
     B-Times       0.58      0.85      0.69        13
     B-Units       0.84      0.68      0.75        38
       I-Age       1.00      1.00      1.00        21
  I-Currency       0.62      0.50      0.55        26
     I-Dates       0.93      0.96      0.94        53
    I-Prices       0.33      0.43      0.38        21
  I-Quantity       0.50      0.67      0.57         3
     I-Times       0.52      0.91      0.66        43
     I-Units       0.91      0.62      0.73        65
           O       0.97      0.96      0.97       483

    accuracy                           0.87       870
   macro avg       0.75   

{'eval_loss': 0.5348508954048157,
 'eval_done': 1,
 'eval_runtime': 2.8626,
 'eval_samples_per_second': 20.611,
 'eval_steps_per_second': 10.48}

In [5]:
from app.inference import infer

In [6]:
infer(["I will go to work at 5:00 AM and my car is red"], model_tuned=model_tuned, model_main=model_main, tokenizer_main=tokenizer_main, tokenizer_tuned=tokenizer_tuned, max_length=64)

{'main': [{'5:00': {'index': [6], 'entity': 'time'},
   'red': {'index': [12], 'entity': 'artwork'}}],
 'tuned': [{'5 : 00 am': {'index': [6, 7, 8, 9], 'entity': 'Times'}}],
 'regex': [{'5:00 AM': {'start_idx': 21, 'end_idx': 28, 'entity': 'Time'},
   'red': {'start_idx': 43, 'end_idx': 46, 'entity': 'Color'}}]}

In [2]:
from app.regex_finders import regex_ner

In [3]:
regex_ner("I will go to work at 5:00 AM and my car is red at the speed of 50 km/hr")

{'5:00 AM': {'start_idx': 21, 'end_idx': 28, 'entity': 'Time'},
 'red': {'start_idx': 43, 'end_idx': 46, 'entity': 'Color'},
 'km': {'start_idx': 66, 'end_idx': 68, 'entity': 'Unit'}}